<a href="https://colab.research.google.com/github/mistryvivek/A-Level-NEA/blob/main/Model_1_Basic_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import torch
import torch.nn as nn
import numpy as np
import random
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import OneHotEncoder

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


# Setup Weights and Bias.

* AI experimental tool.

In [3]:
!pip install wandb -qU

In [4]:
# Log in to your W&B account
import wandb
import random
import math

In [5]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mistry-vivek03 (mistry-vivek03-university-of-york). Use `wandb login --relogin` to force relogin


True

# Make RNN

In [6]:
class RNN(nn.Module):
    def __init__(self,max_laps,hidden_size):
        super(RNN, self).__init__()
        self.embedding =  nn.Embedding(max_laps, hidden_size)
        self.linear_h = nn.Linear(hidden_size,hidden_size)
        self.linear_y = nn.Linear(hidden_size,max_laps)
        self.tanh = nn.Tanh()

    def forward(self,x,hprev):
        h = self.tanh(self.embedding(x) + self.linear_h(hprev))
        y = self.linear_y(h)
        return h,y

# Loss function (Based upon wk11 practical)

* Vocab size is not relevent here - going to give it binary classification values anyway.
* Everything else can be applied for time series: use Cross Entropy Loss, Length of the race = seq_len.
* For each lap, pass through the RNN with previous hidden state and build up hte matrix of outputs.
* Detact the previous hidden state as a new one will be generated.

In [154]:
def calculate_loss(model,inputs,targets,hprev,max_tyre_life):
  loss_func = nn.BCEWithLogitsLoss()
  seq_length = len(inputs)
  #outputs = torch.zeros(0, max_laps)
  #print(outputs.shape)
  for t in range(seq_length):
    # For each character in the input sequence, pass through RNN with previous hidden state
    hprev,y = model(inputs[t], hprev)
    # Gradually build up matrix of output logits of size seq_length * max_tyre_life
    #outputs = torch.cat((outputs ,y))

  y = y[-1]

  # Compute cross entropy loss for seq_length actual targets against estimated distributions
  loss = loss_func(y,targets.squeeze(0))

  # For truncated backprop, the next subsequence will use the final hidden state
  # but will not backprop through it so we need to detach
  hprev = hprev.detach()

  return loss, hprev

# My dataset - how to format?

* Training and testing
* inputs_training/ inputs_testing/  outputs_testing/ output_training
* For now, it will all just be a 2d array.

ERROR:

```2551     return torch.embedding(weight, input, padding_idx, scale_grad_by_freq, sparse)
   2552
   2553

IndexError: index out of range in self```

In [155]:
BASE_PATH = r"/content/drive/MyDrive/prbx_data/v1/"
max_race_size = 0
max_tyre_life = 0

In [156]:
# Define compound categories and one-hot encode them
compound_categories = ['SOFT', 'MEDIUM', 'HARD', 'WET', 'INTERMEDIATE']
compound_encoder = OneHotEncoder(categories=[compound_categories], sparse_output=False)

In [157]:
# TRAINING
training_inputs = []
training_outputs = []

RaceCalender22 = pd.read_csv(BASE_PATH + r"2022/eventCalender2022.csv")
for _, row in RaceCalender22.iterrows():
    if row['EventFormat'] != 'testing':
        # Load race data
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2022/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            # Filter data for the specific driver and sort by LapNumber
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            """if max(TempRaceLoadDriver['TyreLife']) > max_tyre_life:
                max_tyre_life = max(TempRaceLoadDriver['TyreLife'])"""

            # Extract tyre life and stint columns
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values
            compound_array = TempRaceLoadDriver['Compound'].values
            compound_encoded = compound_encoder.fit_transform(compound_array.reshape(-1, 1))

            # Create the stint change array
            stint_change_array = []
            for i in range(len(stint_array)):
                if i == len(stint_array) - 1:  # Last lap
                    stint_change_array.append(False)  # Treat as no change
                else:
                    stint_change_array.append(stint_array[i] != stint_array[i + 1])

            stint_change_array = np.array(stint_change_array)

            # Ensure tyre_life_array and stint_change_array are properly formatted
            tyre_life_array = torch.tensor(tyre_life_array, dtype=torch.long, device=device)
            compound_encoded = torch.tensor(compound_encoded, dtype=torch.long, device=device)
            stint_change_array = torch.tensor(stint_change_array, dtype=torch.float32, device=device)

            combined_inputs = torch.cat([tyre_life_array.unsqueeze(1), compound_encoded], dim=1)

            # Add to training data
            training_inputs.append(combined_inputs)
            training_outputs.append(stint_change_array)

In [158]:
# TESTING
testing_inputs = []
testing_outputs = []

RaceCalender23 = pd.read_csv(BASE_PATH + r"2023/eventCalender2023.csv")
for _, row in RaceCalender23.iterrows():
    if row['EventFormat'] != 'testing':
        # Load race data
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2023/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            # Filter data for the specific driver and sort by LapNumber
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            if max(TempRaceLoadDriver['TyreLife']) > max_tyre_life:
                max_tyre_life = max(TempRaceLoadDriver['TyreLife'])

            # Extract tyre life and stint columns
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values
            compound_array = TempRaceLoadDriver['Compound'].values
            compound_encoded = compound_encoder.fit_transform(compound_array.reshape(-1, 1))

            # Create the stint change array
            stint_change_array = []
            for i in range(len(stint_array)):
                if i == len(stint_array) - 1:  # Last lap
                    stint_change_array.append(False)  # Treat as no change
                else:
                    stint_change_array.append(stint_array[i] != stint_array[i + 1])

            stint_change_array = np.array(stint_change_array)

            # Ensure tyre_life_array and stint_change_array are properly formatted
            tyre_life_array = torch.tensor(tyre_life_array, dtype=torch.long, device=device)
            compound_encoded = torch.tensor(compound_encoded, dtype=torch.long, device=device)
            stint_change_array = torch.tensor(stint_change_array, dtype=torch.float32, device=device)

            combined_inputs = torch.cat([tyre_life_array.unsqueeze(1), compound_encoded], dim=1)

            # Add to testing data
            testing_inputs.append(combined_inputs)
            testing_outputs.append(stint_change_array)

# Training the model

* https://developers.google.com/machine-learning/crash-course/overfitting/interpreting-loss-curves

In [186]:
hidden_size = 50
lr = 0.001
iterations = 10000
max_race_size = int(max_race_size)
input_parameters = ['TyreLife', 'Compound']
dataset = 'v1'

In [187]:
def evaluate_model(inputs, outputs, tolerance=3):
    total_correct = 0
    total_predictions = 0

    for seq_idx in range(len(inputs)):
        # Initialize hidden state for each sequence
        hprev = torch.zeros(hidden_size, device=device)

        # Get the input and target sequences
        input = inputs[seq_idx]
        output = outputs[seq_idx].cpu().numpy()

        # Step through the input sequence and get predicted probabilities
        for t in range(len(input)):
            hprev, y = model(torch.tensor(input[t], device=device), hprev)
            predicted_pits = torch.sigmoid(y) # Apply sigmoid to logits to get probabilities

        y = y[-1]

        predicted_pits = predicted_pits.cpu().detach().numpy()[0]
        predicted_pits = predicted_pits > 0.5
        actual_pits = output == 1.0

        # Now check accuracy for this sequence with tolerance
        correct_count = 0
        total_predictions_for_sequence = 0

        for predicted_pit_idx in range(len(predicted_pits)):
          if predicted_pits[predicted_pit_idx]:  # If a pit stop is predicted
              total_predictions += 1

              # Calculate the tolerance window around the predicted pit
              start_idx = max(0, predicted_pit_idx - tolerance)
              end_idx = min(len(actual_pits), predicted_pit_idx + tolerance + 1)

              # Check if there's any actual pit stop within this window using NumPy
              if np.any(actual_pits[start_idx:end_idx] == 1):  # If there's a pit stop in this window
                  correct_count += 1
                  break

        total_correct += correct_count

    # Calculate overall accuracy
    accuracy = total_correct / total_predictions if total_predictions > 0 else 0
    return accuracy * 100

In [188]:
def pad_with_last_value(tensor, max_race_size):
    if len(tensor) >= max_race_size:
        return tensor

    last_value = tensor[-1]

    # TO FIX: RuntimeError: a Tensor with 6 elements cannot be converted to Scalar
    # After adding compound
    if last_value.ndimension() > 0:
        padding = last_value.unsqueeze(0).repeat(max_race_size - len(tensor), 1)
    else:
        padding = torch.full((max_race_size - len(tensor),), last_value.item(), dtype=tensor.dtype, device=device)

    return torch.cat((tensor, padding))

In [ ]:
model = RNN(max_race_size, hidden_size)
model = model.to(device)

optim = torch.optim.Adam(model.parameters(), lr = lr)

permutation = torch.randperm(len(training_inputs))
training_inputs = [training_inputs[i] for i in permutation]
training_outputs = [training_outputs[i] for i in permutation]

# Initialize WandB
wandb.init(
    project="prbx-model-v1",
    config={
        "learning_rate": lr,
        "iterations": iterations,
        "hidden_size": hidden_size,
        "max_race_size": max_race_size,
        "input_parameters": input_parameters,
        "dataset": dataset,
        "optimizer": "Adam",
        "random_perms": True,
    }
)

n, p = 0, 0
while n<=iterations:
  hprev = torch.zeros(hidden_size,) # reset RNN memory

  inputs = training_inputs[p]
  targets = training_outputs[p]

  inputs = pad_with_last_value(inputs, max_race_size).to(device)
  targets = pad_with_last_value(targets, max_race_size).to(device)
  hprev = torch.zeros(hidden_size, device=device)

  # Compute loss for current subsequence
  loss, hprev = calculate_loss(model,inputs,targets,hprev,max_race_size)
  optim.zero_grad()
  loss.backward()
  optim.step()

  if n % 500 == 0:
    print('[{:}] Loss: {:.2f}'.format(n,loss.item()))
    testing_accuracy = evaluate_model(testing_inputs,testing_outputs)
    training_accuracy = evaluate_model(training_inputs,training_outputs)

    # Log metrics to WandB
    wandb.log({
        "iteration": n,
        "loss": loss.item(),
        "training_accuracy": training_accuracy,
        "testing_accuracy": testing_accuracy,
    })

    # Save to weights and biases.
    model_path = "model.pth"
    torch.save(model.state_dict(), model_path)


    artifact = wandb.Artifact("basic-rnn-binary-classifier", type="model")
    artifact.add_file(model_path)
    wandb.log_artifact(artifact)

  n += 1 # iteration counter
  p += 1 # dataset counter

  #Restart to the front of the list.
  if len(training_inputs) - 1 == p:
    p = 0
    permutation = torch.randperm(len(training_inputs))
    training_inputs = [training_inputs[i] for i in permutation]
    training_outputs = [training_outputs[i] for i in permutation]

iteration,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
loss,█▂▁▂▂▂▂▁▄▃▁▃▁▄▃▁▃▃▂▂▂
testing_accuracy,▄▁█▆▁▆▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_accuracy,▄▁█▅▁▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁
iteration,10000
loss,0.10264
testing_accuracy,0
training_accuracy,0


[0] Loss: 0.76


<ipython-input-187-3f4cf054cb99>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  hprev, y = model(torch.tensor(input[t], device=device), hprev)


[500] Loss: 0.08
[1000] Loss: 0.02
[1500] Loss: 0.09
[2000] Loss: 0.15
[2500] Loss: 0.06
[3000] Loss: 0.10
[3500] Loss: 0.14
[4000] Loss: 0.15
[4500] Loss: 0.03
[5000] Loss: 0.06
[5500] Loss: 0.07
[6000] Loss: 0.11
[6500] Loss: 0.05
[7000] Loss: 0.11
[7500] Loss: 0.11
[8000] Loss: 0.04
[8500] Loss: 0.05
[9000] Loss: 0.06
[9500] Loss: 0.07


#How can we calculate accuracy for this basic model?

* Say is probability is above 80% percent pit, otherwise don't pit.
* Want at least one right pit stop identification to class it as match.

In [15]:
for t in range(len(testing_inputs[1])):
  # For each character in the input sequence, pass through RNN with previous hidden state
  hprev,y = model(torch.tensor([testing_inputs[1][t]], device=device),hprev)
probabilities = torch.sigmoid(y)

In [16]:
probabilities

tensor([[1.8245e-02, 2.9221e-02, 2.3090e-02, 5.2584e-02, 1.4823e-03, 9.7559e-05,
         3.1790e-02, 6.2055e-02, 1.6202e-02, 1.4026e-02, 1.0448e-01, 1.8828e-01,
         2.0165e-01, 2.3096e-01, 1.1071e-01, 4.6514e-03, 8.8460e-03, 2.7448e-02,
         6.0488e-02, 7.9396e-03, 4.9523e-02, 1.1347e-02, 4.5724e-02, 3.3656e-03,
         2.4717e-02, 4.2204e-02, 4.4635e-02, 3.3482e-03, 3.6820e-03, 1.8051e-02,
         3.3702e-01, 6.6892e-02, 1.7792e-01, 1.3327e-01, 1.8120e-01, 5.4868e-02,
         9.0611e-03, 9.4199e-02, 6.4338e-03, 8.8866e-02, 7.0160e-02, 3.2610e-02,
         1.0926e-01, 1.9035e-02, 3.3946e-02, 1.3522e-02, 2.4385e-01, 1.4249e-01,
         7.9675e-04, 1.0232e-01, 5.6250e-02, 2.3119e-01, 1.0167e-02, 1.0459e-01,
         5.0600e-01, 2.7306e-01, 1.1362e-01, 1.1651e-02, 3.5161e-05, 3.7102e-05,
         3.7390e-05, 3.0720e-05, 3.4394e-05, 4.5009e-05, 3.2237e-05, 3.6523e-05,
         7.5240e-04, 4.4399e-05, 1.9246e-03, 4.3706e-05, 5.9389e-05, 4.3656e-05,
         5.2367e-05, 3.7198e

In [17]:
for t in range(len(training_inputs[0])):
  # For each character in the input sequence, pass through RNN with previous hidden state
  hprev,y = model(torch.tensor([training_inputs[0][t]], device=device),hprev)
probabilities = torch.sigmoid(y)

In [18]:
probabilities

tensor([[4.2579e-01, 1.7864e-02, 9.4681e-03, 1.4281e-04, 1.5162e-01, 5.4269e-03,
         2.9113e-03, 1.1016e-04, 1.4941e-01, 2.7708e-03, 9.5839e-03, 2.1853e-02,
         1.7905e-02, 2.0627e-02, 8.2723e-02, 4.7361e-02, 7.4057e-03, 3.8286e-02,
         1.7217e-01, 3.0808e-02, 1.2943e-05, 5.4943e-02, 9.6523e-02, 1.1784e-01,
         2.0476e-01, 1.6795e-02, 2.0350e-03, 5.6160e-04, 5.7783e-02, 1.1220e-01,
         1.5802e-03, 3.4600e-02, 1.1455e-01, 2.0244e-02, 2.5614e-03, 8.1257e-03,
         5.5695e-02, 3.8255e-02, 4.6864e-01, 3.6415e-03, 4.3508e-04, 1.3077e-04,
         7.5569e-03, 9.1153e-03, 7.3553e-02, 1.4736e-02, 1.3291e-02, 4.8492e-03,
         1.9084e-03, 3.7519e-03, 1.6917e-02, 1.2566e-03, 2.7801e-03, 2.1017e-02,
         2.7895e-04, 1.0429e-02, 1.0856e-03, 5.1355e-03, 5.2290e-06, 5.7702e-06,
         5.5054e-06, 5.9562e-06, 5.1920e-06, 5.6708e-06, 5.5325e-06, 6.1476e-06,
         1.2775e-04, 6.6866e-06, 5.4204e-04, 4.0143e-06, 6.3745e-06, 5.6179e-06,
         8.4090e-06, 5.4181e

# Plot to see what this basic model has remembered - probably just averages

# Validating my data

In [19]:
len(training_inputs) == len(training_outputs)

True

In [20]:
training_inputs[0].shape

torch.Size([69])

In [21]:
training_outputs[0].shape

torch.Size([69])